## Data preparation

In [1]:
import numpy as np
import scipy
import os
import sklearn as skl
import pandas as pd
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import pylab as plt
from pandas.plotting import scatter_matrix
#import librosa  #need install first
#import librosa.display

In [3]:
# Load metadata and features - change to local directory for tracks file
tracks = pd.read_csv('/Users/rulanxiao/Desktop/fma_metadata/tracks.csv', header=None)
echonest = pd.read_csv('/Users/rulanxiao/Desktop/fma_metadata/echonest.csv', header=None)

/Users/rulanxiao/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/rulanxiao/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,11,13,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175

In [ ]:
# Change column names from "track.1", ... to named columns
header = tracks.iloc[1]
header[0] = 'track_ID'
tracks.drop(tracks.index[[0,1,2]], inplace=True)
tracks.rename(columns=header, inplace=True)
tracks.head()

header = echonest.iloc[2]
header[0]='track_ID'
echonest.drop(echonest.index[[0,1,2,3]],inplace=True)
echonest.rename(columns = header,inplace=True)
echonest.head()

# Only select songs for which we have echonest data
tracks_with_echonest_data = tracks[tracks['track_ID'].isin(echonest['track_ID'])]
echonest_data = echonest[echonest['track_ID'].isin(tracks_with_echonest_data['track_ID'])]

merged_echonest_data = pd.merge(tracks_with_echonest_data, echonest_data, on = 'track_ID')

# Change duplicate "listens" column to track listens and album listens
duplicate_listens = {'listens': ['album_listens', 'track_listens']}

merged_echonest_data = merged_echonest_data.rename(columns=lambda c: duplicate_listens[c].pop(0) if c in duplicate_listens.keys() else c)

In [ ]:
# Extract year released from release date variable
album_release_year = []

for x in merged_echonest_data['date_released']:
    if type(x)==str:
        album_release_year.append(x[0:4])
    else: 
        album_release_year.append(np.nan)

#add album release year to dataframe
merged_echonest_data['album_release_year'] = album_release_year

# Check number of songs released per year and pick year with maximum
import collections
counter=collections.Counter(album_release_year)
print(counter)
print(counter.values())
print(counter.keys())
print(counter.most_common(3))

In [ ]:
# For chosen year, check the number of songs released per genre; pick top three genres to use
tracks_2010 = merged_echonest_data[merged_echonest_data['album_release_year'] == '2010']

# Top genres for 2010
counter=collections.Counter(tracks_2010['genre_top'])
print(counter)
print(counter.values())
print(counter.keys())
print(counter.most_common(10))

In [ ]:
# Add sentiment analysis by title into dataset - compound score of positive / negative sentiment for song title
# Rulan
# require to install nltk first
# this part work nice, file will be generated to our project folder. 

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# Read in track data - change to local directory
tracks_senti = pd.read_csv('tracks.csv',header=None)

header = tracks_senti.iloc[1]
header[0]='track ID'
header[52]='track title'
tracks_senti.drop(tracks_senti.index[[0,1,2]],inplace=True)
tracks_senti.rename(columns = header,inplace=True)
tracks_senti.head()

df=tracks_senti[['track ID','track title']]
df.dropna(axis=0, how='any')

ml = df["track title"].values
title=[]
for i in range(len(ml)):
    a=str(ml[i])
    title.append(a)
idd = df["track ID"].values

neg=[]
neu=[]
pos=[]
comp=[]

sid = SentimentIntensityAnalyzer()
for sentence in title:
    ss = sid.polarity_scores(sentence)
    score=[]
    for k in ss:
        a=ss[k]
        score.append(a)
    neg.append(score[0])
    neu.append(score[1])
    pos.append(score[2])
    comp.append(score[3])

sentimentall = pd.DataFrame({'track_ID':idd,'track_title':title,'senti neg': neg,'senti neu': neu,'senti pos': pos,'senti comp': comp})

sentimentall.to_csv('sentimental_analysis_title.csv')

In [ ]:
# Add sentiment to dataframe
sentiment= pd.read_csv('sentimental_analysis_title.csv')
sentiment=sentiment[['track_ID','senti comp', 'senti neg','senti pos']]
tracks_2010 = pd.merge(tracks_2010, sentiment, how='inner', on=['track_ID'])
tracks_2010.head()

In [ ]:
# Select the variables we want - named variables 
t10 = tracks_2010[['track_ID','latitude','longitude','bit_rate','duration','genre_top','track_listens','acousticness','danceability','energy','instrumentalness','liveness','speechiness','tempo','valence','artist_hotttnesss','artist_discovery','artist_familiarity', 'senti neg','senti pos']]
t10['popular'] = (t10['track_listens']>=1147.25).astype(int)
# For location (latitude, longitude), if na, change to 0
t10['latitude'].fillna(0, inplace=True)
t10['longitude'].fillna(0, inplace=True)

# For genre-specific models - split dataset into three based on top three genres
Hiphop_10=t10.loc[t10['genre_top'] == 'Hip-Hop']
Rock_10=t10.loc[t10['genre_top'] == 'Rock']
Elec_10=t10.loc[t10['genre_top'] == 'Electronic']

# For baseline model - add genre as a categorical variable 
t10['Rock'] = (t10['genre_top'] == 'Rock').astype(int)
t10['Hip-Hop'] = (t10['genre_top'] == 'Hip-Hop').astype(int)
t10['Electronic'] = (t10['genre_top'] == 'Electronic').astype(int)
t10.drop(['genre_top'], axis=1, inplace=True)

t10.head()

In [ ]:
# Take hip-hop, rock, and hip-hop together
t10['g']=t10['Rock']+ t10['Hip-Hop'] +t10['Electronic']
t10=t10.loc[t10['g'] == 1]
t10.drop(['g'],axis=1,inplace=True)

# only genres Hiphop, Rock and Electronic (separate dataframes)
Hiphop_10.drop(['genre_top'],axis=1,inplace=True)
Rock_10.drop(['genre_top'],axis=1,inplace=True)
Elec_10.drop(['genre_top'],axis=1,inplace=True)

In [ ]:
Hiphop_10.head()

In [ ]:
# Make boxplot of variables for t10 data (data with all three genres) - before standardization 
t10.drop(['track_ID'], axis=1).astype(float).boxplot()
plt.title('Boxplot of variables for dataset with three genres, before standardization')
plt.xticks(rotation=90)
plt.show()

# Make boxplot of variables for t10 data (data with all three genres) - after standardization 
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
t10_scaled = scaler.fit_transform(t10.drop(['track_ID'], axis=1))
t10_scaled = pd.DataFrame(t10_scaled)
t10_scaled.rename(columns={'0': 'latitude', '1':'longitude'}, inplace=True)

t10_scaled.astype(float).boxplot()
plt.title('Boxplot of variables for dataset with three genres, after standardization')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Make boxplot of variables for rock data - before standardization 
Rock_10.drop(['track_ID'], axis=1).astype(float).boxplot()
plt.title('Boxplot of variables for dataset with Rock, before standardization')
plt.xticks(rotation=90)
plt.show()

# Make boxplot of variables for rock data - after standardization 
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
Rock_10_scaled = scaler.fit_transform(Rock_10.drop(['track_ID'], axis=1))
Rock_10_scaled = pd.DataFrame(Rock_10_scaled)
Rock_10_scaled.rename(columns={'0': 'latitude', '1':'longitude'}, inplace=True)

Rock_10_scaled.astype(float).boxplot()
plt.title('Boxplot of variables for dataset with Rock, after standardization')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Make boxplot of variables for hiphop data - before standardization 
Hiphop_10.drop(['track_ID'], axis=1).astype(float).boxplot()
plt.title('Boxplot of variables for dataset with Hip-hop, before standardization')
plt.xticks(rotation=90)
plt.show()

# Make boxplot of variables for hiphop data - after standardization 
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
Hiphop_10_scaled = scaler.fit_transform(Rock_10.drop(['track_ID'], axis=1))
Hiphop_10_scaled = pd.DataFrame(Hiphop_10_scaled)
Hiphop_10_scaled.rename(columns={'0': 'latitude', '1':'longitude'}, inplace=True)

Hiphop_10_scaled.astype(float).boxplot()
plt.title('Boxplot of variables for dataset with Hip-hop, after standardization')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Make boxplot of variables for electronic data - before standardization 
Elec_10.drop(['track_ID'], axis=1).astype(float).boxplot()
plt.title('Boxplot of variables for dataset with Electronic, before standardization')
plt.xticks(rotation=90)
plt.show()

# Make boxplot of variables for electronic data - after standardization 
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
Elec_10_scaled = scaler.fit_transform(Elec_10.drop(['track_ID'], axis=1))
Elec_10_scaled = pd.DataFrame(Elec_10_scaled)
Elec_10_scaled.rename(columns={'0': 'latitude', '1':'longitude'}, inplace=True)

Elec_10_scaled.astype(float).boxplot()
plt.title('Boxplot of variables for dataset with Electronic, after standardization')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Make boxplots of variables in genre dataset 
# Sa

# Standardize variables as needed
# Sa
t10.iloc[1]

## Exploratory analysis

In [ ]:
# Check for correlation between variables with correlation plot 
# Kathy

# Check for correlation between arist hotness, familiarity, discovery
# Kathy
rock10

In [ ]:
# Make scatter plots of numerical variables versus outcome (track listens)
# Rulan

# an outlier on those categories


rock10=t10[t10['genre_top']=='Rock']
rockm=rock10.apply(pd.to_numeric, errors='coerce')
rockm=rockm.drop(['track_ID','Rock','Hip-Hop','Electronic','genre_top'],axis=1)
rock102=rockm.drop(['latitude','longitude','duration','bit_rate','artist_hotttnesss','artist_discovery','artist_familiarity'],axis=1)
rock101=rockm[['track_listens','latitude','longitude','duration','bit_rate','artist_hotttnesss','artist_discovery','artist_familiarity']]


scatter_matrix(rock101,figsize=(20,20))
plt.show()


In [ ]:
scatter_matrix(rock102,figsize=(30,30))
plt.show()

In [ ]:

plt.scatter(rockm.track_listens,rockm.acousticness)
plt.ylabel('track_listens')
plt.xlabel('duration')
plt.title('track_listens v.s. duration')
plt.show()

plt.scatter(rockm.track_listens,rockm.duration)
plt.ylabel('track_listens')
plt.xlabel('acousticness')
plt.title('track_listens v.s. acousticness')
plt.show()

plt.scatter(rockm.track_listens,rockm.energy)
plt.ylabel('track_listens')
plt.xlabel('energy')
plt.title('track_listens v.s. energy')
plt.show()

plt.scatter(rockm.track_listens,rockm.instrumentalness)
plt.ylabel('track_listens')
plt.xlabel('instrumentalness')
plt.title('track_listens v.s. instrumentalness')
plt.show()

plt.scatter(rockm.track_listens,rockm.liveness)
plt.ylabel('track_listens')
plt.xlabel('liveness')
plt.title('track_listens v.s. liveness')
plt.show()

plt.scatter(rockm.track_listens,rockm.speechiness)
plt.ylabel('track_listens')
plt.xlabel('speechiness')
plt.title('track_listens v.s. speechiness')
plt.show()

plt.scatter(rockm.track_listens,rockm.tempo)
plt.ylabel('track_listens')
plt.xlabel('tempo')
plt.title('track_listens v.s. tempo')
plt.show()


In [ ]:

# Correlation plots for categorical variables

## Model selection

In [ ]:
# Baseline model - use the year with most songs data and include genre as a categorical variable (use the same top 3 genres)

# Convert genre to dummy variables

# Use linear regression with same variables as above (sentiment analysis, track metadata, echonest named variable, location)

# IDEA: we will see that genre is a big predictor of track listens - thus, we train separate models for each genre to dig deeper into why that is
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

y = t10[['popular']].values
X=t10.drop(['track_listens','track_ID','popular'],axis=1).values
scaler = preprocessing.StandardScaler()
X_scaled = scaler.fit_transform(X)
#y_scaled = scaler.fit_transform(y.reshape(-1, 1))


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=7)


linear = LinearRegression()
linear.fit(X_train, y_train)

linear_score=linear.score(X_test, y_test)

print(linear_score)

importances = pd.DataFrame({'feature':t10.drop(['track_listens','track_ID','popular'],axis=1).columns,'importance':linear.coef_[0]})
plt.bar(linear.coef_[0])
plt.xticks(np.arange(len(regr.coef_[0])),t10.drop(['track_listens','track_ID','popular'],axis=1).columns, rotation=90)
print(importances.sort_values(by='importance'))

In [ ]:
# Baseline model - use the year with most songs data and include genre as a categorical variable (use the same top 3 genres)

# Convert genre to dummy variables

# Use linear regression with same variables as above (sentiment analysis, track metadata, echonest named variable, location)

# IDEA: we will see that genre is a big predictor of track listens - thus, we train separate models for each genre to dig deeper into why that is

# TRY CLASSIFICATION - songs with top 10% track listens are POPULAR, else NOT POPULAR

# Convert track listens to categorical (if track_listens > 1147.25, then = 1)


In [ ]:
# Function for plotting ROC
def generate_ROCplot(fpr,tpr,label,roc_auc):
    plt.clf()
    plt.plot(fpr, tpr, '.-',label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
# Train logistic regression for popularity

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = t10['popular']
X=t10.drop(['track_listens','popular', 'track_ID'],axis=1).values
scaler = preprocessing.StandardScaler()
X_scaled = scaler.fit_transform(X)
#y_scaled = scaler.fit_transform(y.reshape(-1, 1))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)


regr = LogisticRegression(penalty='l1')
regr.fit(X_train, y_train)

#compute AUC ROC
import sklearn.metrics
from sklearn.metrics import roc_curve, auc

#compute ROC 
probas_ = regr.fit(X_train,y_train).predict_proba(X_test)

# Generate ROC  for LR with l1 penalty
fpr, tpr, thresholds = roc_curve(y_test, probas_[:, 1])
roc_auc = auc(fpr, tpr)
print("Area under the ROC curve : %f" % roc_auc)

# Plots ROC
generate_ROCplot(fpr,tpr,'LR',roc_auc)

In [ ]:
# Plot coefficient importance
plt.bar(np.arange(len(regr.coef_[0])),regr.coef_[0])
plt.xticks(np.arange(len(regr.coef_[0])), ['latitude', 'longitude', 'bit_rate', 'duration',
       'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'speechiness', 'tempo', 'valence',
       'artist_hotttnesss', 'artist_discovery', 'artist_familiarity',
       'senti neg', 'senti pos', 'Rock', 'Hip-Hop', 'Electronic'], rotation=90)
plt.show()

In [ ]:
# For 5-folds, split data into training and testing (one of the genres) - will repeat for other two genres (maybe write function to do this)

# Train linear regression model using all variables

# Regularize using L^1 penalty due to large number of features - pick optimal penalty and compare R^2

# Use GridSearchCV with at least 2-fold validation

# Check resulting model on 5 random folds of data 

# Plot coefficients for resulting model 


In [ ]:
# Train random forest regression model 

# Regularize using L^1 penalty due to large number of features - pick optimal penalty and compare R^2

# Use GridSearchCV with at least 2-fold validation

# Check resulting model on 5 random folds of data 

# Plot coefficients for resulting model 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

y = Rock_10[['popular']].values
X=Rock_10.drop(['track_listens','track_ID'],axis=1).values
scaler = preprocessing.StandardScaler()
X_scaled = scaler.fit_transform(X)
#y_scaled = scaler.fit_transform(y.reshape(-1, 1))


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=7)


random_forest = RandomForestRegressor(n_estimators=100)
random_forest.fit(X_train, y_train)

Y_prediction = random_forest.predict(X_test)

random_forest.score(X_test, y_test)
acc_random_forest = random_forest.score(X_test, y_test) * 100
print(acc_random_forest)
importances = pd.DataFrame({'feature':Rock_10.drop(['track_listens','track_ID'],axis=1).columns,'importance':random_forest.feature_importances_})
importances = importances.sort_values('importance',ascending=False).set_index('feature')
importances.plot.bar()

In [ ]:
# Explore adding other features to model, such as nonlinear features (interaction terms)

In [ ]:
# Rock

In [ ]:
# Jazz

In [ ]:
# Hip hop

In [ ]:
# Compare how models are different between different genres - for instance, if certain variables are more or less important for different genres

In [ ]:
# Write function that takes in optimal_model, new song audio and necessary info (like genre, track title, etc.) and uses librosa to extract features
# and output a prediction of number of track listens 

# for testing a new song, im not sure if our model gonna use "features", so i'll also look for data in "echonest "

# librosa code only extract same data on features

# Load the example clip
# Load 30 seconds of a wav file, starting 15 seconds in
y, sr = librosa.load('blahblah.mp3', offset=15.0, duration=30.0)

# Set the hop length; at 22050 Hz, 512 samples ~= 23ms
hop_length = 512

# Separate harmonics and percussives into two waveforms
y_harmonic, y_percussive = librosa.effects.hpss(y)

# Beat track on the percussive signal
tempo, beat_frames = librosa.beat.beat_track(y=y_percussive,
                                             sr=sr)

# Compute MFCC features from the raw signal
mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)

# And the first-order differences (delta features)
mfcc_delta = librosa.feature.delta(mfcc)

# Stack and synchronize between beat events
# This time, we'll use the mean value (default) instead of median
beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]),
                                    beat_frames)

# Compute chroma features from the harmonic signal
chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
                                        sr=sr)

# Aggregate chroma features between beat events
# We'll use the median value of each feature between beat frames
beat_chroma = librosa.util.sync(chromagram,
                                beat_frames,
                                aggregate=np.median)

# Finally, stack all beat-synchronous features together
beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

In [ ]:
##Statistic for each feature
def columns():
    feature_sizes = dict(chroma_stft=12, chroma_cqt=12, chroma_cens=12,
                         tonnetz=6, mfcc=20, rmse=1, zcr=1,
                         spectral_centroid=1, spectral_bandwidth=1,
                         spectral_contrast=7, spectral_rolloff=1)
    moments = ('mean', 'std', 'skew', 'kurtosis', 'median', 'min', 'max')

    columns = []
    for name, size in feature_sizes.items():
        for moment in moments:
            it = ((name, moment, '{:02d}'.format(i+1)) for i in range(size))
            columns.extend(it)

    names = ('feature', 'statistics', 'number')
    columns = pd.MultiIndex.from_tuples(columns, names=names)

    # More efficient to slice if indexes are sorted.
    return columns.sort_values()


def feature_stats(name, values):
    features[name, 'mean'] = np.mean(values, axis=1)
    features[name, 'std'] = np.std(values, axis=1)
    features[name, 'skew'] = stats.skew(values, axis=1)
    features[name, 'kurtosis'] = stats.kurtosis(values, axis=1)
    features[name, 'median'] = np.median(values, axis=1)
    features[name, 'min'] = np.min(values, axis=1)
    features[name, 'max'] = np.max(values, axis=1)


features = pd.Series(index=columns(), dtype=np.float32)


In [ ]:
##
y, sr = librosa.load('xxx.mp3', offset=15.0, duration=30.0)

stft=np.abs(librosa.stft(y, n_fft=2048, hop_length=512))
f = librosa.feature.chroma_stft(S=stft**2, n_chroma=12)
feature_stats('chroma_stft', f)

f = librosa.feature.zero_crossing_rate(y, frame_length=2048, hop_length=512)
feature_stats('zcr', f)

cqt = np.abs(librosa.cqt(y, sr=sr, hop_length=512, bins_per_octave=12,
                                 n_bins=7*12, tuning=None))
assert cqt.shape[0] == 7 * 12
assert np.ceil(len(y)/512) <= cqt.shape[1] <= np.ceil(len(y)/512)+1


f = librosa.feature.chroma_cqt(C=cqt, n_chroma=12, n_octaves=7)
feature_stats('chroma_cqt', f)

f = librosa.feature.chroma_cens(C=cqt, n_chroma=12, n_octaves=7)
feature_stats('chroma_cens', f)

f = librosa.feature.tonnetz(chroma=f)
feature_stats('tonnetz', f)

print(features)